In [1]:
import gudhi
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from gudhi.representations import Landscape
import gudhi.representations
from itertools import combinations
import seaborn as sns
from gudhi.hera import wasserstein_distance
from gudhi.hera import bottleneck_distance
from collections import defaultdict
from scipy.interpolate import interp1d
from scipy.stats import ks_2samp
from scipy.spatial.distance import pdist

In [2]:
#Global functions
def nodesExtractor(name): #extracts nodes and their corresponding information
    file_path = 'Complement/' + name +'.csv'
    # Extract the 'connectivity' field from the 'Data' structured array
    points = pd.read_csv(file_path)
    points = points.iloc[:, 1:]
    # Save the DataFrame to inspect it
    return points

def nodesToArray(name):
    nodes_df = nodesExtractor(name)
    nodes_loc = nodes_df.loc[:,['x','y','z']]
    loc_array = nodes_loc.to_numpy()
    return loc_array

def compute_persistence(points):
    """
    Compute persistence diagram and track the last death in each dimension.
    Excludes infinite values in H1 and H2.
    """
    # --- Build alpha complex ---
    alpha_complex = gudhi.AlphaComplex(points=points)
    simplex_tree = alpha_complex.create_simplex_tree()
    simplex_tree.compute_persistence()
    
    # --- Collect persistence pairs ---
    persistence_pairs = simplex_tree.persistence()
    
    # DIAGNOSTIC: Check for infinite bars
    for dim, (birth, death) in persistence_pairs:
        if np.isinf(death):
            print(f"WARNING: Infinite bar in H{dim}: birth={birth}, death=∞")
    
    # --- Filter infinite bars for H1/H2 ---
    diag = []
    for dim, (birth, death) in persistence_pairs:
        diag.append((dim, (birth, death)))
    
    # --- Track last death in each dimension ---
    last_deaths = {}
    for dim, (birth, death) in diag:
        if np.isfinite(death):
            if dim not in last_deaths:
                last_deaths[dim] = death
            else:
                last_deaths[dim] = max(last_deaths[dim], death)
    
    return diag, last_deaths

def average_curves(curve_dicts, resolution=100000):
    """
    Generic averaging function for any type of curve dicts (Betti, lifespan, etc.)
    curve_dicts: list of dicts {dim: (grid, curve)}
    Returns: dict {dim: (common_grid, avg_curve)}
    """
    average = {}

    dims = set().union(*(d.keys() for d in curve_dicts))

    for dim in dims:
        all_grids, all_curves = [], []
        for cdict in curve_dicts:
            if dim not in cdict:
                continue
            grid, curve = cdict[dim]
            if len(grid) == 0:
                continue
            all_grids.append(grid)
            all_curves.append(curve)

        if not all_grids:
            average[dim] = (np.array([]), np.zeros(resolution))
            continue

        global_min = min(g[0] for g in all_grids)
        global_max = max(g[-1] for g in all_grids)
        common_grid = np.linspace(global_min, global_max, resolution)

        interpolated = []
        for grid, curve in zip(all_grids, all_curves):
            f = interp1d(grid, curve, bounds_error=False, fill_value=0)
            interpolated.append(f(common_grid))

        avg_curve = np.mean(interpolated, axis=0)
        average[dim] = (common_grid, avg_curve)

    return average

def plot_curves(curves_dicts, labels, dimension, title='Betti Curves Comparison', limit = [0,100]):
    """
    curves_dicts: list of betti_curve dicts {dim: (grid, curve)}
    labels: list of labels for each curve
    dimension: the Betti number (int) to plot
    """
    plt.figure(figsize=(10, 6))

    colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'cyan', 'magenta']
    linestyles = ['-', '--', '-.', ':']

    for idx, (curve_dict, label) in enumerate(zip(curves_dicts, labels)):
        if dimension in curve_dict:
            grid, curve = curve_dict[dimension]
            if len(grid) > 0:
                color = colors[idx % len(colors)]
                linestyle = linestyles[idx % len(linestyles)]
                plt.plot(grid, curve, color=color, linestyle=linestyle, label=f'{label} (Dimension-{dimension})')
        else:
            print(f"Warning: {label} does not contain Betti-{dimension}")

    plt.title(f'{title} (Betti-{dimension})', fontsize=28)
    plt.xlabel('Filtration Value', fontsize=20)
    plt.ylabel(f'Betti-{dimension}', fontsize=20)
    plt.xlim(limit)
    plt.legend(fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.grid(False)
    plt.tight_layout()
    plt.show()

def save_curves(curves_dicts, labels, dimension, filename, title='Betti Curves Comparison', limit=[0, 100], dpi=300):
    """
    curves_dicts: list of betti_curve dicts {dim: (grid, curve)}
    labels: list of labels for each curve
    dimension: the Betti number (int) to plot
    filename: path to save the plot (e.g., 'output.png' or 'output.jpg')
    dpi: resolution of saved image (default 300)
    """
    plt.figure(figsize=(10, 6))
    colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'cyan', 'magenta']
    linestyles = ['-', '--', '-.', ':']
    
    for idx, (curve_dict, label) in enumerate(zip(curves_dicts, labels)):
        if dimension in curve_dict:
            grid, curve = curve_dict[dimension]
            if len(grid) > 0:
                color = colors[idx % len(colors)]
                linestyle = linestyles[idx % len(linestyles)]
                plt.plot(grid, curve, color=color, linestyle=linestyle, label=f'{label} (Dimension-{dimension})')
        else:
            print(f"Warning: {label} does not contain Betti-{dimension}")
    
    plt.title(f'{title} (Betti-{dimension})', fontsize=28)
    plt.xlabel('Filtration Value', fontsize=20)
    plt.ylabel(f'Betti-{dimension}', fontsize=20)
    plt.xlim(limit)
    plt.legend(fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(filename, dpi=dpi, bbox_inches='tight')
    plt.close()  # Close the figure to free memory
    
    print(f"Saved plot to {filename}")

In [3]:
pressure = '4'
lobe = 'left'

datasets = {
    'm1053007': nodesToArray('m1p'+pressure+'_053007_'+lobe),
    'm2053007': nodesToArray('m2p'+pressure+'_053007_'+lobe),
    'm1053107': nodesToArray('m1p'+pressure+'_053107_'+lobe),
    'm2053107': nodesToArray('m2p'+pressure+'_053107_'+lobe),
    'm1060107': nodesToArray('m1p'+pressure+'_060107_'+lobe),
    'm1060407': nodesToArray('m1p'+pressure+'_060407_'+lobe),
    'm2060407': nodesToArray('m2p'+pressure+'_060407_'+lobe),
    'm3060407': nodesToArray('m3p'+pressure+'_060407_'+lobe),
    'm1060507': nodesToArray('m1p'+pressure+'_060507_'+lobe),
    'm2060507': nodesToArray('m2p'+pressure+'_060507_'+lobe),
    'm3060507': nodesToArray('m3p'+pressure+'_060507_'+lobe),
    'm2060607': nodesToArray('m2p'+pressure+'_060607_'+lobe),
    'm3060607': nodesToArray('m3p'+pressure+'_060607_'+lobe)
}

# Example usage across all datasets
persistence_results = {
    name: compute_persistence(points)
    for name, points in datasets.items()
}

# Separate results
persistence_diagrams = {name: res[0] for name, res in persistence_results.items()}
last_deaths = {name: res[1] for name, res in persistence_results.items()}

In [4]:
# Compute global max death across all datasets for each dimension
global_last_deaths = {}
for name, deaths_by_dim in last_deaths.items():
    for dim, death in deaths_by_dim.items():
        if dim not in global_last_deaths:
            global_last_deaths[dim] = death
        else:
            global_last_deaths[dim] = max(global_last_deaths[dim], death)

print("Global max deaths by dimension:")
for dim in sorted(global_last_deaths.keys()):
    print(f"  H{dim}: {global_last_deaths[dim]:.4f}")

Global max deaths by dimension:
  H0: 0.2471
  H1: 1.6614
  H2: 1.9275


# Betti Curve

In [5]:
def compute_betti_curve(diag, max_dim=2, resolution=100000, cutoff=None):
    betti_curves = {}
    
    # Determine global filtration range across ALL dimensions
    all_values = []
    for dim, (birth, death) in diag:
        all_values.append(birth)
        if np.isfinite(death):
            all_values.append(death)
    
    global_min = np.min(all_values) if all_values else 0
    global_max = np.max(all_values) if all_values else 1
    
    if cutoff is not None:
        global_max = cutoff
    
    for dim in range(max_dim + 1):
        diag_dim = np.array([pt[1] for pt in diag if pt[0] == dim])
        
        if len(diag_dim) == 0:
            betti_curves[dim] = (np.zeros(resolution), np.zeros(resolution))
            continue
        
        # Use GLOBAL range, not per-dimension range
        grid = np.linspace(global_min, global_max, resolution)
        curve = np.zeros_like(grid)
        
        for birth, death in diag_dim:
            if np.isinf(death):
                # Infinite bar: alive from birth onward
                curve += (grid >= birth)
            else:
                # Finite bar: alive in [birth, death]
                curve += (grid >= birth) & (grid <= death)
        
        betti_curves[dim] = (grid, curve)
    
    return betti_curves

In [6]:
betti_curves = {}
for name, diag in persistence_diagrams.items():
    points = datasets[name]
    betti_curves[name] = compute_betti_curve(diag, cutoff=global_last_deaths[dim])

KeyboardInterrupt: 

In [ ]:
# Get keys in insertion order
keys = list(betti_curves.keys())

hyper_betti_curves = [betti_curves[k] for k in keys[:5]]
control_betti_curves  = [betti_curves[k] for k in keys[-8:]]

average_betti_control = average_curves(control_betti_curves)
average_betti_hyper = average_curves(hyper_betti_curves)

In [ ]:
#Plots Betti curves for one mouse
"""for dim in np.arange(3):    
    plot_curves(
        [hyper_betti_curves[4]],
        labels=["m1053007"], dimension = dim,
        title="Betti Curve", limit = [0,global_last_deaths[dim]]
    )"""

'for dim in np.arange(3):    \n    plot_curves(\n        [hyper_betti_curves[4]],\n        labels=["m1053007"], dimension = dim,\n        title="Betti Curve", limit = [0,global_last_deaths[dim]]\n    )'

In [ ]:
#Plots Average Betti Curves
"""for dim in np.arange(3):    
    plot_curves(
        [average_betti_hyper, average_betti_control],
        labels=["Hyper", "Control"], dimension = dim,
        title="Comparison of Average Betti Curves", limit = [0,global_last_deaths[dim]]
    )"""

'for dim in np.arange(3):    \n    plot_curves(\n        [average_betti_hyper, average_betti_control],\n        labels=["Hyper", "Control"], dimension = dim,\n        title="Comparison of Average Betti Curves", limit = [0,global_last_deaths[dim]]\n    )'

# Persistence Landscapes

In [ ]:
def compute_landscape_curves(
    persistence_diagram,
    t_min,
    t_max,
    max_dim=2,
    k=5,
    resolution=500
):
    """
    Compute persistence landscapes for all dimensions.
    Returns dict of {dim: (t_vals, landscapes)} where landscapes is shape (k, resolution)
    """
    landscape_curves = {}
    
    for dim in range(max_dim + 1):
        diagram = np.array([
            (birth, death)
            for d, (birth, death) in persistence_diagram
            if d == dim and np.isfinite(death)  # Filter out infinite bars
        ])
        
        if diagram.shape[0] == 0:
            # No features in this dimension
            t_vals = np.linspace(t_min, t_max, resolution)
            landscapes = np.zeros((k, resolution))
            landscape_curves[dim] = (t_vals, landscapes)
            continue
        
        landscape = Landscape(
            num_landscapes=k,
            resolution=resolution,
            sample_range=(t_min, t_max)
        )
        
        # IMPORTANT: input must be a list of NumPy arrays
        landscapes = landscape.fit_transform([diagram])[0]
        landscapes = landscapes.reshape(k, resolution)
        t_vals = np.linspace(t_min, t_max, resolution)
        
        landscape_curves[dim] = (t_vals, landscapes)
    
    return landscape_curves


def save_landscape(t_vals, landscapes, dim, filename, title=None, limit=[0, 100], dpi=300):
    """
    Save persistence landscape plot to file.
    landscapes: shape (k, resolution) array of k landscape functions
    """
    plt.figure(figsize=(10, 6))
    
    for i, layer in enumerate(landscapes):
        plt.plot(t_vals, layer, label=f"$\\lambda_{{{i+1}}}$")
    
    plt.xlabel("Filtration Value", fontsize=20)
    plt.ylabel("Landscape Value", fontsize=20)
    plt.title(title or f"Persistence Landscape (H_{dim})", fontsize=28)
    plt.xlim(limit)
    plt.legend(fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.grid(False)
    plt.tight_layout()
    
    plt.savefig(filename, dpi=dpi, bbox_inches='tight')
    plt.close()
    
    print(f"Saved landscape plot to {filename}")

def plot_landscape(t_vals, landscapes, dim, title=None):
    plt.figure(figsize=(8, 5))
    for i, layer in enumerate(landscapes):
        plt.plot(t_vals, layer, label=f"$\\lambda_{{{i+1}}}$")

    plt.xlabel("t")
    plt.ylabel("Landscape value")
    plt.title(title or f"Persistence Landscape (H_{dim})")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Compute landscapes for all datasets
landscape_curves = {}
for name, diag in persistence_diagrams.items():
    landscape_curves[name] = compute_landscape_curves(
        diag,
        t_min=0,
        t_max=max(global_last_deaths.values()),
        max_dim=2,
        k=3,
        resolution=500
    )


# Lifespan Curves

In [ ]:
def extract_intervals(persistence_diagram, dim):
    return np.array([
        (b, d)
        for d_dim, (b, d) in persistence_diagram
        if d_dim == dim and np.isfinite(d)
    ])

def compute_lifespan_curves(
    persistence_diagram,
    t_min,
    t_max,
    max_dim=2,
    resolution=500
):
    lifespan_curves = {}
    
    for dim in range(max_dim + 1):
        intervals = extract_intervals(persistence_diagram, dim)
        
        if intervals.size == 0:
            lifespan_curves[dim] = (np.zeros(resolution), np.zeros(resolution))
            continue
        
        births = intervals[:, 0]
        deaths = intervals[:, 1]
        lifespans = deaths - births
        
        # Use GLOBAL range, not per-dimension range
        t_vals = np.linspace(t_min, t_max, resolution)
        LC = np.zeros_like(t_vals)
        
        for i, t in enumerate(t_vals):
            alive = (births <= t) & (t < deaths)
            LC[i] = lifespans[alive].sum()
        
        lifespan_curves[dim] = (t_vals, LC)
    
    return lifespan_curves

In [ ]:
lifespan_curves = {}
for name, diag in persistence_diagrams.items():
    points = datasets[name]
    lifespan_curves[name] = compute_lifespan_curves(diag, t_min = 0, t_max=global_last_deaths[dim], resolution = 100000)

In [ ]:
# Get keys in insertion order
keys = list(lifespan_curves.keys())

hyper_lifespan_curves = [lifespan_curves[k] for k in keys[:5]]
control_lifespan_curves  = [lifespan_curves[k] for k in keys[-8:]]

average_lifespan_control = average_curves(control_lifespan_curves)
average_lifespan_hyper = average_curves(hyper_lifespan_curves)

In [ ]:
# Plots lifespan curves for one mouse
"""for dim in np.arange(3):    
    plot_curves(
        [hyper_lifespan_curves[4]],
        labels=["m1053007"], dimension = dim,
        title="Lifespan Curve", limit = [0,global_last_deaths[dim]]
    )"""

'for dim in np.arange(3):    \n    plot_curves(\n        [hyper_lifespan_curves[4]],\n        labels=["m1053007"], dimension = dim,\n        title="Lifespan Curve", limit = [0,global_last_deaths[dim]]\n    )'

In [ ]:
#Plots average lifespan curves
"""for dim in np.arange(3):    
    plot_curves(
        [average_lifespan_hyper, average_lifespan_control],
        labels=["Hyper", "Control"], dimension = dim,
        title="Comparison of Average Lifespan Curves", limit = [0,global_last_deaths[dim]]
    )"""

'for dim in np.arange(3):    \n    plot_curves(\n        [average_lifespan_hyper, average_lifespan_control],\n        labels=["Hyper", "Control"], dimension = dim,\n        title="Comparison of Average Lifespan Curves", limit = [0,global_last_deaths[dim]]\n    )'

# Norm Lifespan Curves

In [ ]:
def compute_norm_lifespan_curves(
    persistence_diagram,
    t_min,
    t_max,
    max_dim=2,
    resolution=500
):
    lifespan_curves = {}
    
    for dim in range(max_dim + 1):
        intervals = extract_intervals(persistence_diagram, dim)
        
        if intervals.size == 0:
            lifespan_curves[dim] = (np.zeros(resolution), np.zeros(resolution))
            continue
        
        births = intervals[:, 0]
        deaths = intervals[:, 1]
        lifespans = deaths - births
        
        # Compute total lifespan for normalization
        total_lifespan = lifespans.sum()
        
        # Use GLOBAL range, not per-dimension range
        t_vals = np.linspace(t_min, t_max, resolution)
        LC = np.zeros_like(t_vals)
        
        for i, t in enumerate(t_vals):
            alive = (births <= t) & (t < deaths)
            # Normalized: sum of lifespans alive at t / total lifespan
            if total_lifespan > 0:
                LC[i] = lifespans[alive].sum() / total_lifespan
            else:
                LC[i] = 0
        
        lifespan_curves[dim] = (t_vals, LC)
    
    return lifespan_curves

In [ ]:
norm_lifespan_curves = {}
for name, diag in persistence_diagrams.items():
    points = datasets[name]
    norm_lifespan_curves[name] = compute_norm_lifespan_curves(diag, t_min = 0, t_max=global_last_deaths[dim], resolution = 100000)

In [ ]:
# Get keys in insertion order
keys = list(norm_lifespan_curves.keys())

hyper_norm_lifespan_curves = [norm_lifespan_curves[k] for k in keys[:5]]
control_norm_lifespan_curves  = [norm_lifespan_curves[k] for k in keys[-8:]]

average_norm_lifespan_control = average_curves(control_norm_lifespan_curves)
average_norm_lifespan_hyper = average_curves(hyper_norm_lifespan_curves)

In [ ]:
#Plot all lifespan curves for one mouse
"""for dim in np.arange(3):    
    plot_curves(
        [hyper_norm_lifespan_curves[4]],
        labels=[keys[4]], dimension = dim,
        title="Norm Lifespan Curve", limit = [0,global_last_deaths[dim]]
    )"""

'for dim in np.arange(3):    \n    plot_curves(\n        [hyper_norm_lifespan_curves[4]],\n        labels=[keys[4]], dimension = dim,\n        title="Norm Lifespan Curve", limit = [0,global_last_deaths[dim]]\n    )'

In [ ]:
#Plot average lifespan curves across groups in all three dimensions
"""for dim in np.arange(3):    
    plot_curves(
        [average_lifespan_hyper, average_lifespan_control],
        labels=["Hyper", "Control"], dimension = dim,
        title="Comparison of Average Norm Lifespan Curves", limit = [0,global_last_deaths[dim]]
    )"""

'for dim in np.arange(3):    \n    plot_curves(\n        [average_lifespan_hyper, average_lifespan_control],\n        labels=["Hyper", "Control"], dimension = dim,\n        title="Comparison of Average Norm Lifespan Curves", limit = [0,global_last_deaths[dim]]\n    )'

# Save Figures

In [ ]:
#saves all average curves
for dim in np.arange(3):
    filename = '2DComplementGraphs/Pressure'+str(pressure)+'/'+lobe+'/Betti_'+str(dim)+'_Average_P'+str(pressure)+'_'+lobe
    save_curves([average_betti_hyper, average_betti_control], labels=['Hyper','Control'], dimension=dim, filename=filename, 
        title=f"Comparison of Average Betti Curves Lobe {lobe}", limit = [0,global_last_deaths[dim]], dpi=300)

    filename = '2DComplementGraphs/Pressure'+str(pressure)+'/'+lobe+'/Lifespan_'+str(dim)+'_Average_P'+str(pressure)+'_'+lobe
    save_curves([average_lifespan_hyper, average_lifespan_control], labels=['Hyper','Control'], dimension=dim, filename=filename, 
        title=f"Comparison of Average Lifespan Curves Lobe {lobe}", limit = [0,global_last_deaths[dim]], dpi=300)

    filename = '2DComplementGraphs/Pressure'+str(pressure)+'/'+lobe+'/Norm_Lifespan_'+str(dim)+'_Average_P'+str(pressure)+'_'+lobe
    save_curves([average_norm_lifespan_hyper, average_norm_lifespan_control], labels=['Hyper','Control'], dimension=dim, filename=filename, 
        title="Comparison of Average Norm Lifespan Curves", limit = [0,global_last_deaths[dim]], dpi=300)

Saved plot to 2DComplementGraphs/Pressure4/left/Betti_0_Average_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_0_Average_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_0_Average_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Betti_1_Average_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_1_Average_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_1_Average_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Betti_2_Average_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_2_Average_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_2_Average_P4_left


## Betti Curve

In [ ]:
keys = list(betti_curves.keys())
graph = 'Betti'
for dim in range(3):
        for i in range(13):
                mouse_name = keys[i]
                if i< 5:
                        filename = '2DComplementGraphs/Pressure'+str(pressure)+'/'+lobe+'/'+graph+'_'+str(dim)+'_Hyper_'+mouse_name+'_P'+str(pressure)+'_'+lobe
                else:
                        filename = '2DComplementGraphs/Pressure'+str(pressure)+'/'+lobe+'/'+graph+'_'+str(dim)+'_Control_'+mouse_name+'_P'+str(pressure)+'_'+lobe
                save_curves([betti_curves[mouse_name]], labels=[mouse_name], dimension=dim, filename=filename, 
                        title=f"Betti Curve Lobe {lobe}", limit = [0,global_last_deaths[dim]], dpi=300)

Saved plot to 2DComplementGraphs/Pressure4/left/Betti_0_Hyper_m1053007_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Betti_0_Hyper_m2053007_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Betti_0_Hyper_m1053107_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Betti_0_Hyper_m2053107_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Betti_0_Hyper_m1060107_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Betti_0_Control_m1060407_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Betti_0_Control_m2060407_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Betti_0_Control_m3060407_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Betti_0_Control_m1060507_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Betti_0_Control_m2060507_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Betti_0_Control_m3060507_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Betti_0_Control_m2060607_P4_left
Saved plot to 2DComplementGraphs/Press

## Lifespan

In [ ]:

keys = list(lifespan_curves.keys())
graph = 'Lifespan'
for dim in range(3):
        for i in range(13):
                mouse_name = keys[i]
                if i< 5:
                        filename = '2DComplementGraphs/Pressure'+str(pressure)+'/'+lobe+'/'+graph+'_'+str(dim)+'_Hyper_'+mouse_name+'_P'+str(pressure)+'_'+lobe
                else:
                        filename = '2DComplementGraphs/Pressure'+str(pressure)+'/'+lobe+'/'+graph+'_'+str(dim)+'_Control_'+mouse_name+'_P'+str(pressure)+'_'+lobe
                save_curves([lifespan_curves[mouse_name]], labels=[mouse_name], dimension=dim, filename=filename, 
                        title=f"Lifespan Curve Lobe {lobe}", limit = [0,global_last_deaths[dim]], dpi=300)



Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_0_Hyper_m1053007_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_0_Hyper_m2053007_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_0_Hyper_m1053107_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_0_Hyper_m2053107_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_0_Hyper_m1060107_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_0_Control_m1060407_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_0_Control_m2060407_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_0_Control_m3060407_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_0_Control_m1060507_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_0_Control_m2060507_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_0_Control_m3060507_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Lifespan_0_Control_m2060607_P4_left
Sa

## Norm Lifespan

In [ ]:

keys = list(norm_lifespan_curves.keys())
graph = 'Norm_Lifespan'

for dim in range(3):
        for i in range(13):
                mouse_name = keys[i]
                if i< 5:
                        filename = '2DComplementGraphs/Pressure'+str(pressure)+'/'+lobe+'/'+graph+'_'+str(dim)+'_Hyper_'+mouse_name+'_P'+str(pressure)+'_'+lobe
                else:
                        filename = '2DComplementGraphs/Pressure'+str(pressure)+'/'+lobe+'/'+graph+'_'+str(dim)+'_Control_'+mouse_name+'_P'+str(pressure)+'_'+lobe
                save_curves([norm_lifespan_curves[mouse_name]], labels=[mouse_name], dimension=dim, filename=filename, 
                        title=f"Norm Lifespan Curve Lobe {lobe}", limit = [0,global_last_deaths[dim]], dpi=300)



Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_0_Hyper_m1053007_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_0_Hyper_m2053007_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_0_Hyper_m1053107_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_0_Hyper_m2053107_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_0_Hyper_m1060107_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_0_Control_m1060407_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_0_Control_m2060407_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_0_Control_m3060407_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_0_Control_m1060507_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_0_Control_m2060507_P4_left
Saved plot to 2DComplementGraphs/Pressure4/left/Norm_Lifespan_0_Control_m3060507_P4_left
Saved plot to 2DComplementGraph

## Persistence Landscapes

In [ ]:
keys = list(landscape_curves.keys())
graph = 'Persistence_Landscape'
for i in range(13):
    mouse_name = keys[i]
    for dim in [0, 1, 2]:
        if i< 5:
            filename = '2DComplementGraphs/Pressure'+str(pressure)+'/'+lobe+'/'+graph+'_'+str(dim)+'_Hyper_'+mouse_name+'_P'+str(pressure)+'_'+lobe
        else:
                filename = '2DComplementGraphs/Pressure'+str(pressure)+'/'+lobe+'/'+graph+'_'+str(dim)+'_Control_'+mouse_name+'_P'+str(pressure)+'_'+lobe
        t_vals, landscapes = landscape_curves[mouse_name][dim]
        save_landscape(
            t_vals,
            landscapes,
            dim=dim,
            filename=filename,
            title=f'Persistence Landscape {name} Dimension {dim} Lobe {lobe}',
            limit=[0, global_last_deaths[dim]],
            dpi=150
        )

Saved landscape plot to 2DComplementGraphs/Pressure4/left/Persistence_Landscape_0_Hyper_m1053007_P4_left
Saved landscape plot to 2DComplementGraphs/Pressure4/left/Persistence_Landscape_1_Hyper_m1053007_P4_left
Saved landscape plot to 2DComplementGraphs/Pressure4/left/Persistence_Landscape_2_Hyper_m1053007_P4_left
Saved landscape plot to 2DComplementGraphs/Pressure4/left/Persistence_Landscape_0_Hyper_m2053007_P4_left
Saved landscape plot to 2DComplementGraphs/Pressure4/left/Persistence_Landscape_1_Hyper_m2053007_P4_left
Saved landscape plot to 2DComplementGraphs/Pressure4/left/Persistence_Landscape_2_Hyper_m2053007_P4_left
Saved landscape plot to 2DComplementGraphs/Pressure4/left/Persistence_Landscape_0_Hyper_m1053107_P4_left
Saved landscape plot to 2DComplementGraphs/Pressure4/left/Persistence_Landscape_1_Hyper_m1053107_P4_left
Saved landscape plot to 2DComplementGraphs/Pressure4/left/Persistence_Landscape_2_Hyper_m1053107_P4_left
Saved landscape plot to 2DComplementGraphs/Pressure4/le